# Segmenting and Clustering Neighborhoods in Toronto part I
1.Create Jupiter Notebook & import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [2]:
print('Libraries imported.')

Libraries imported.


In [3]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

In [4]:
import folium

In [5]:
from bs4 import BeautifulSoup

2.Scrape Wikipedia page

In [10]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(data, 'html.parser')


In [41]:
right_table=soup.find('table',class_='wikitable sortable')


In [42]:
all_tables = soup.find_all('table')

3.Create dataframe

* The dataframe consist of three columns: Postcode, Borough, Neighborhood

In [43]:
postcode = []
borough = []
neighborhood = []

In [44]:
for row in right_table.findAll('tr'):
    
    cells= row.findAll('td')
    
    if len(cells) ==3:
        postcode.append(cells[0].find(text=True))
        borough.append(cells[1].find(text=True))
        neighborhood.append(cells[2].find(text=True))
        
        

In [45]:
df = pd.DataFrame({'Postcode': postcode,
                       'Borough': borough,
                       'Neighborhood': neighborhood})

In [46]:
df.head()



,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


* Drop Non assigned values in 'Borough' column

In [53]:
df.drop(df[df.Borough.str.contains('Not assigned')].index,inplace =True)

In [54]:
df = df.reset_index(drop=True)

In [55]:
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


* Grouping Neighborhood

In [59]:
df.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ",".join (x))

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge\n"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek\n"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n
5,M1J,Scarborough,Scarborough Village\n
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park\n"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge\n"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West\n"
9,M1N,Scarborough,"Birch Cliff, Cliffside West\n"


* Checking Non assigned value for Neighborhood

In [37]:
df.Neighborhood.str.contains('Not assigned')

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
30     False
31     False
32     False
33     False
34     False
35     False
36     False
37     False
38     False
39     False
40     False
41     False
42     False
43     False
44     False
45     False
46     False
47     False
48     False
49     False
50     False
51     False
52     False
53     False
54     False
55     False
56     False
57     False
58     False
59     False
60     False
61     False
62     False
63     False
64     False
65     False
66     False
67     False
68     False
69     False
70     False
71     False
72     False
73     False
74     False
75     False
76     False

* use the .shape method to print the number of rows of your dataframe

In [40]:
df.shape

(103, 3)

# Segmenting and Clustering Neighborhoods in Toronto part II

* geting the latitude and the longitude coordinates of each neighborhood

In [63]:
url = 'http://cocl.us/Geospatial_data'

In [66]:
gecod = pd.read_csv(url)

In [67]:
gecod.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


* useing the Geocoder package or the csv file to create the following dataframe

In [77]:
gecod.rename(columns={'Postal Code': 'Postcode'}, inplace=True)

In [79]:
gecod.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [82]:
Bor_gecod = pd.merge(df, gecod, on = 'Postcode', how = 'inner')

In [83]:
Bor_gecod.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
